# Explorando convolução no Keras usando o Theano

O objetivo deste notebook é de explorar a convolução que é implementada pelo Theano através do Keras.

A convolução é a operação essencial das redes convolucionais. É uma operação demorada e a sua implementação na GPU acelera bastante o seu processamento. A implementação é feita no Theano utilizando GPU. Se a GPU não for encontrada, a implementação roda na CPU.

A figura a seguir ilustra uma convolução da imagem verde de entrada, utilizando o núcleo da convolução (kernel) em amarelo, deslizando sobre a imagem verde. O resultado da convolução é a imagem rosa, de tamanho ligeiramente menos, devido ao processamento na borda da imagem.

Esta ilustração é implementada no código que chama a convolução do Theano mais à frente.

![](http://mourafiq.com/images/posts/convolution_schematic.gif)

# Primeira solução - usando o backend do Theano

In [31]:
import numpy as np
import keras
from keras.models import Sequential
from keras.backend import theano_backend as KTH
from keras.utils.np_utils import convert_kernel



## formato do shape da imagem: th ou tf
- th: theano - [canal, altura, largura]
- tf: tensor flow - [altura, largura, canal]


O trecho abaixo é uma ilustração da chamada da função de convolução do Theano. A implementação da convolução é de 4 dimensões, tanto a imagem como o núcleo (kernel da convolução).

É necessária utilizar a interface *backend* do Keras para acessar o Theano

In [32]:
x_val = np.array([[[[1,1,1,0,0],
                    [0,1,1,1,0],
                    [0,0,1,1,1], 
                    [0,0,1,1,0],
                    [0,1,1,0,0]]]])

print 'Imagem de entrada: x_val:\n', x_val

x_th = KTH.variable(x_val)

kernel_val = np.array([[[[1,0,1],
                         [0,1,0],
                         [1,0,1]]]])
print 'Núcleo da convolução kernel_val:\n', kernel_val

kernel_th = KTH.variable(convert_kernel(kernel_val, dim_ordering='th'))

z_th = KTH.eval(KTH.conv2d(x_th, kernel_th, dim_ordering='th',border_mode='valid'))

print 'Resultado z_th:\n', z_th


Imagem de entrada: x_val:
[[[[1 1 1 0 0]
   [0 1 1 1 0]
   [0 0 1 1 1]
   [0 0 1 1 0]
   [0 1 1 0 0]]]]
Núcleo da convolução kernel_val:
[[[[1 0 1]
   [0 1 0]
   [1 0 1]]]]
Resultado z_th:
[[[[ 4.  3.  4.]
   [ 2.  4.  3.]
   [ 2.  3.  4.]]]]


# 2a. Solução - usando o predict do pipeline e inicializando os pesos




In [33]:
from keras.models import Sequential
from keras.layers.core import Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
np.set_printoptions(suppress=True, precision=3)

W = 1./9. * np.ones((3,3)).reshape(1,1,3,3)

# apply a 3x3 convolution with 1 output filters on a 11x11 image:
model = Sequential()
model.add(Convolution2D(1, 3, 3, weights=[W], border_mode='same', input_shape=(1, 11, 11),bias=False))
# now model.output_shape == (None, 1, 256, 256)

#model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
#model.add(Activation("relu"))

x = np.zeros((11,11))
x[5,5]=1
x = x.reshape(1,1,11,11)
y = model.predict(x,batch_size=1)
print y.shape
print y

print len(model.layers)
w = model.layers[0].get_weights()

print w

(1, 1, 11, 11)
[[[[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
   [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
   [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
   [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
   [ 0.     0.     0.     0.     0.111  0.111  0.111  0.     0.     0.     0.   ]
   [ 0.     0.     0.     0.     0.111  0.111  0.111  0.     0.     0.     0.   ]
   [ 0.     0.     0.     0.     0.111  0.111  0.111  0.     0.     0.     0.   ]
   [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
   [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
   [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
   [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]]]]
1
[array([[[[ 0.111,  0.111,  0.111],
         [ 0.111,  0.111,  0.111],
       

# 3a. solução - Usando a API funcional do Keras

## Exercícios

### Teórico

1. Supondo uma imagem de entrada 100 x 100 e um kernel 3 x 3. Utilizando-se o border_mode como válido, a imagem de saída será de 99 x 99. Se fosse utilizar uma rede densa, com entrada de 100 x 100 e saída 99 x 99, quantos parâmetros seriam necessários para ser treinados? E no caso da rede convolucional? Qual é o fator de redução?

### Práticos

1. Trocar o parâmetro ``border_mode`` para ``same`` e veja a diferença.
2. Rodar a convolução para uma imagem de cinza
3. Rodar a convolução para uma imagem colorida RGB
4. Rodar a convolução para gerar várias bandas de saída como nas redes convolucionais
5. Comparar o tempo de execução desta convolução com outras implementações: openCV por exemplo